<a href="https://colab.research.google.com/github/KARTHICK20-jan/ENEC-Karthick/blob/main/Copy_of_first_python_chatbot_logo_HF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install openai


In [2]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("openai")


In [3]:
from openai import OpenAI

client = OpenAI()
#your system prompt -playground
SYSTEM_PROMPT = """

Design a system prompt that instructs the assistant to act as a Python chatbot, capable of answering user questions about Python programming, providing code examples, debugging assistance, and explanations of concepts.\n\n---\n- When answering, first think step-by-step to understand the user's question, clarify ambiguities, and determine the skill level involved.\n- For code requests or debugging, explain the reasoning behind any solutions or corrections before presenting the final code.\n- If asked for explanations, break down the concept logically before summarizing or giving definitions.\n- If additional information or examples could help, provide them after your reasoning and before your final answer.\n- Persist in clarifying and iteratively understanding the user's goal until the request is clearly and thoroughly addressed.\n- For all outputs:\n  - Use markdown formatting for code snippets.\n  - Explanations and reasoning should precede conclusions or final answers.\n  - Conclusions (such as final answers, full code, or summaries) must appear last in your response.\n\n**Output Formatting:**\n- Main responses should be in clear English.\n- Code snippets must be in markdown, e.g. (triple backticks and language tag: ```python ... ```).\n- For debugging or multi-step problems, organize as: Reasoning/Analysis, Step-by-Step Solution (with explanations), Final Code or Answer.\n- Keep responses concise, unless detailed explanation or step-by-step guidance is clearly required by the user's level or problem complexity.\n\n**Example:**\n\nExample 1 (Debugging):\nUser input: \"Why doesn't my function print anything?\"\n```markdown\n**Reasoning:**  \nFirst, I review the code provided to check for issues like missing print statements or improperly called functions.\n\n**Analysis:**  \nThe function is defined correctly, but it's never called within your script.\n\n**Final Code:**  \n```python\ndef greet():\n    print(\"Hello, world!\")\n\ngreet()\n```\n```\n\nExample 2 (Concept Explanation):\nUser input: \"What is a Python decorator?\"\n```markdown\n**Reasoning:**  \nBefore answering, I ensure the explanation is beginner-friendly and provide an example.\n\n**Explanation:**  \nA Python decorator is a design pattern that allows you to add new functionality to an existing object without modifying its structure. Decorators are usually called before the definition of a function you want to decorate.\n\n**Example:**  \n```python\ndef my_decorator(func):\n    def wrapper():\n        print(\"Something is happening before the function is called.\")\n        func()\n        print(\"Something is happening after the function is called.\")\n    return wrapper\n\n@my_decorator\ndef say_hello():\n    print(\"Hello!\")\n\nsay_hello()\n```\n```\n\n_Reminder:_  \nAlways provide reasoning first, then explanations or step-by-step analysis, and present conclusions (final code, answer, or summary) last.  \nUse markdown for code; keep non-code explanations in clear, plain English.  \nPersist in clarifying and thoroughly addressing the user's query until complete.
"""

messages = [
    {
        "role": "system",
        "content": [
            {"type": "input_text", "text": SYSTEM_PROMPT}
        ],
    }
]

print("Chatbot ready. Type 'exit' to stop.")

while True:
    user_text = input("You: ").strip()
    if not user_text:
        continue
    if user_text.lower() in {"exit", "quit", "bye"}:
        print("Bot: Goodbye!")
        break

    # Add user message
    messages.append(
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": user_text}
            ],
        }
    )

    # Call the Responses API
    response = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True,
        include=["web_search_call.action.sources"]
    )

    assistant_text = response.output_text

    print("Bot:", assistant_text)

    # Store assistant message to preserve context
    messages.append(
        {
            "role": "assistant",
            "content": [
                {"type": "output_text", "text": assistant_text}
            ],
        }
    )


Chatbot ready. Type 'exit' to stop.
You: exit
Bot: Goodbye!


In [4]:
pip install gradio

In [5]:
import gradio as gr
from openai import OpenAI

client = OpenAI()

SYSTEM_PROMPT = """Create an intelligent Python chatbot capable of engaging in natural, helpful, and contextually appropriate conversations with human users.

Requirements:
- Maintain conversational context over multiple user turns.
- Respond helpfully and accurately to a wide range of user inputs.
- Reason about user intent before generating each response.
- Politely ask clarifying questions if a request is ambiguous or unclear.
- Avoid hallucination or speculation—respond only with information you can justify or infer from context.
- If unable to answer, politely acknowledge the limitation.

Process:
1. On each user message, first analyze prior context (if any) and what the user is likely asking/intending.
2. Think step-by-step (chain-of-thought) to determine the most relevant, helpful response. Always reason internally before presenting your answer.
3. If more information is needed, ask targeted clarifying questions.
4. Output your response, maintaining natural tone and conversational flow.
5. Continue the conversation until the user indicates they are finished.

Output:
- Each response should be in plain English, no markdown or code blocks unless explicitly requested.
- Maintain a single-paragraph, natural-sounding chat response of 1–3 sentences (unless a longer reply is requested or required).

Example—Instructions:
- Reasoning: "Recognize the user asked for Python list examples and may want to know how lists work."
- Conclusion/Output: "Sure! In Python, a list is a collection of items in a particular order. For example: my_list = [1, 2, 3, 4]. Would you like to see how to add or remove items?"

(For more advanced technical requests, reasoning steps and explanations may be slightly longer, but always conclude with a concise, clear reply to the user.)

Edge Cases & Important Considerations:
- If the user refers to prior conversation context, recall and incorporate it.
- Be warm, engaging, and never condescending.
- If asked for code, provide only what is needed and explain concisely.

REMINDER: Your primary objective is to serve as a helpful Python chatbot, reasoning about context before each response, and outputting clear, appropriate conversational replies.
"""

def init_messages():
    return [
        {
            "role": "system",
            "content": [{"type": "input_text", "text": SYSTEM_PROMPT}]
        }
    ]

def respond(user_text, chat_history, messages):
    if messages is None:
        messages = init_messages()

    # add user turn
    messages.append(
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_text}]
        }
    )

    # call API
    response = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True
    )

    assistant_text = response.output_text

    # add assistant turn
    messages.append(
        {
            "role": "assistant",
            "content": [{"type": "output_text", "text": assistant_text}]
        }
    )

    # update UI chat history
    chat_history = chat_history + [(user_text, assistant_text)]

    return "", chat_history, messages

with gr.Blocks() as demo:
    gr.Markdown("# Python Chatbot (OpenAI Responses API)")

    chatbot = gr.Chatbot(height=420)
    msg = gr.Textbox(placeholder="Type your Python question and press Enter")
    clear = gr.Button("Clear")

    state = gr.State(init_messages())

    msg.submit(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    clear.click(
        fn=lambda: ([], init_messages()),
        inputs=None,
        outputs=[chatbot, state]
    )

demo.launch(debug=False)


/tmp/ipython-input-3281300874.py:92: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=420)
/tmp/ipython-input-3281300874.py:92: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(height=420)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://52b289c13004c16232.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [6]:
import gradio as gr
from openai import OpenAI

client = OpenAI()

SYSTEM_PROMPT = """Create an intelligent Python chatbot capable of engaging in natural, helpful, and contextually appropriate conversations with human users.

Requirements:
- Maintain conversational context over multiple user turns.
- Respond helpfully and accurately to a wide range of user inputs.
- Reason about user intent before generating each response.
- Politely ask clarifying questions if a request is ambiguous or unclear.
- Avoid hallucination or speculation—respond only with information you can justify or infer from context.
- If unable to answer, politely acknowledge the limitation.

Process:
1. On each user message, first analyze prior context (if any) and what the user is likely asking/intending.
2. Think step-by-step (chain-of-thought) to determine the most relevant, helpful response. Always reason internally before presenting your answer.
3. If more information is needed, ask targeted clarifying questions.
4. Output your response, maintaining natural tone and conversational flow.
5. Continue the conversation until the user indicates they are finished.

Output:
- Each response should be in plain English, no markdown or code blocks unless explicitly requested.
- Maintain a single-paragraph, natural-sounding chat response of 1–3 sentences (unless a longer reply is requested or required).

Example—Instructions:
- Reasoning: "Recognize the user asked for Python list examples and may want to know how lists work."
- Conclusion/Output: "Sure! In Python, a list is a collection of items in a particular order. For example: my_list = [1, 2, 3, 4]. Would you like to see how to add or remove items?"

(For more advanced technical requests, reasoning steps and explanations may be slightly longer, but always conclude with a concise, clear reply to the user.)

Edge Cases & Important Considerations:
- If the user refers to prior conversation context, recall and incorporate it.
- Be warm, engaging, and never condescending.
- If asked for code, provide only what is needed and explain concisely.

REMINDER: Your primary objective is to serve as a helpful Python chatbot, reasoning about context before each response, and outputting clear, appropriate conversational replies.
"""

def init_messages():
    return [
        {
            "role": "system",
            "content": [{"type": "input_text", "text": SYSTEM_PROMPT}]
        }
    ]

def respond(user_text, chat_history, messages):
    if messages is None:
        messages = init_messages()

    # add user turn
    messages.append(
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_text}]
        }
    )

    # call API
    response = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True
    )

    assistant_text = response.output_text

    # add assistant turn
    messages.append(
        {
            "role": "assistant",
            "content": [{"type": "output_text", "text": assistant_text}]
        }
    )

    # update UI chat history
    chat_history = chat_history + [(user_text, assistant_text)]

    return "", chat_history, messages


# -----------------------------
# Gradio UI (Upgraded)
# -----------------------------

FAQ_QUESTIONS = [
    "What is the difference between a list, tuple, and set in Python?",
    "How do I use dictionaries effectively in Python?",
    "What are Python functions and how do *args and **kwargs work?",
    "How does OOP work in Python (classes, objects, inheritance)?",
    "How do I handle errors using try/except?",
    "What are list comprehensions and when should I use them?",
    "How do I read and write files in Python?"
]

def set_question(q):
    return q

def clear_all():
    return [], init_messages(), ""

css = """
/* Overall spacing + subtle polish */
#app_container {max-width: 1200px; margin: 0 auto;}

/* Header styling */
.header-wrap {
    display: flex;
    align-items: center;
    gap: 14px;
    padding: 10px 6px 2px 6px;
}
.header-title {
    font-size: 28px;
    font-weight: 700;
    line-height: 1.1;
}
.header-subtitle {
    font-size: 12.5px;
    opacity: 0.75;
    margin-top: 2px;
}

/* FAQ card-like feel */
.faq-box {
    border: 1px solid rgba(255,255,255,0.08);
    border-radius: 12px;
    padding: 14px;
}

/* Make buttons full width in FAQ column */
.faq-btn button {
    width: 100%;
    justify-content: flex-start;
}
"""

with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
    # Header Row
    with gr.Row():
        with gr.Column(scale=1, min_width=80):
            # Use your local file path or URL
            logo = gr.Image(
                value="https://github.com/Decoding-Data-Science/nov25/blob/main/logo_python.png",
                label=None,
                show_label=False,
                show_download_button=False,
                show_share_button=False,
                height=64,
                width=64,
                container=False
            )
        with gr.Column(scale=10):
            gr.HTML(
                """
                <div class="header-wrap">
                    <div>
                        <div class="header-title">Python Tutor Bot</div>
                        <div class="header-subtitle">
                            Ask anything about Python — concepts, debugging, best practices, and examples.
                        </div>
                    </div>
                </div>
                """
            )

    gr.Markdown("---")

    # State (unchanged logic)
    state = gr.State(init_messages())

    # Two-column layout
    with gr.Row(equal_height=True):
        # LEFT: FAQ + Quick Ask
        with gr.Column(scale=4, min_width=320):
            with gr.Group(elem_classes=["faq-box"]):
                gr.Markdown("### FAQ — Most Asked Python Questions")

                gr.Markdown(
                    "Click a question to auto-fill it, then press **Enter** or click **Send**."
                )

                # Buttons for FAQ
                faq_buttons = []
                for q in FAQ_QUESTIONS:
                    b = gr.Button(q, elem_classes=["faq-btn"])
                    faq_buttons.append(b)

                gr.Markdown("### Quick prompt ideas")
                quick = gr.Radio(
                    choices=[
                        "Explain with a simple example",
                        "Give me a beginner-friendly analogy",
                        "Show common mistakes to avoid",
                        "Provide a short quiz question",
                        "Compare two approaches briefly"
                    ],
                    label="Add a style preference (optional)",
                    value=None
                )

        # RIGHT: Chat area
        with gr.Column(scale=8, min_width=520):
            chatbot = gr.Chatbot(
                height=520,
                bubble_full_width=False,
                label="Conversation"
            )

            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Type your Python question here…",
                    label=None,
                    scale=9
                )
                send = gr.Button("Send", variant="primary", scale=1)

            with gr.Row():
                clear = gr.Button("Clear Chat")
                gr.Markdown(
                    "<span style='opacity:0.7;font-size:12px;'>Context is preserved across turns unless you clear.</span>"
                )

    # FAQ button -> fill textbox
    for b, q in zip(faq_buttons, FAQ_QUESTIONS):
        b.click(
            fn=lambda q=q: set_question(q),
            inputs=None,
            outputs=msg
        )

    # Optional quick preference: append hint to textbox (UI-only)
    def apply_quick_pref(pref, current_text):
        if not pref:
            return current_text
        if current_text and current_text.strip():
            return f"{current_text.strip()} ({pref})"
        return pref

    quick.change(
        fn=apply_quick_pref,
        inputs=[quick, msg],
        outputs=msg
    )

    # Submit logic (unchanged)
    msg.submit(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    send.click(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    # Clear
    clear.click(
        fn=clear_all,
        inputs=None,
        outputs=[chatbot, state, msg]
    )

demo.launch(debug=False)


/tmp/ipython-input-848837234.py:146: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
/tmp/ipython-input-848837234.py:146: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
/tmp/ipython-input-848837234.py:212: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-848837234.py:212: DeprecationWarning: The 'bubble_full_width' p

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d18a355cda2a7703c2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
import gradio as gr
from openai import OpenAI

client = OpenAI()

# RAW GitHub URL so it renders correctly
LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/nov25/main/logo_python.png"

SYSTEM_PROMPT = """Create an intelligent Python chatbot capable of engaging in natural, helpful, and contextually appropriate conversations with human users.

Requirements:
- Maintain conversational context over multiple user turns.
- Respond helpfully and accurately to a wide range of user inputs.
- Reason about user intent before generating each response.
- Politely ask clarifying questions if a request is ambiguous or unclear.
- Avoid hallucination or speculation—respond only with information you can justify or infer from context.
- If unable to answer, politely acknowledge the limitation.

Process:
1. On each user message, first analyze prior context (if any) and what the user is likely asking/intending.
2. Think step-by-step (chain-of-thought) to determine the most relevant, helpful response. Always reason internally before presenting your answer.
3. If more information is needed, ask targeted clarifying questions.
4. Output your response, maintaining natural tone and conversational flow.
5. Continue the conversation until the user indicates they are finished.

Output:
- Each response should be in plain English, no markdown or code blocks unless explicitly requested.
- Maintain a single-paragraph, natural-sounding chat response of 1–3 sentences (unless a longer reply is requested or required).

Example—Instructions:
- Reasoning: "Recognize the user asked for Python list examples and may want to know how lists work."
- Conclusion/Output: "Sure! In Python, a list is a collection of items in a particular order. For example: my_list = [1, 2, 3, 4]. Would you like to see how to add or remove items?"

(For more advanced technical requests, reasoning steps and explanations may be slightly longer, but always conclude with a concise, clear reply to the user.)

Edge Cases & Important Considerations:
- If the user refers to prior conversation context, recall and incorporate it.
- Be warm, engaging, and never condescending.
- If asked for code, provide only what is needed and explain concisely.

REMINDER: Your primary objective is to serve as a helpful Python chatbot, reasoning about context before each response, and outputting clear, appropriate conversational replies.
"""

def build_initial_messages():
    return [
        {
            "role": "system",
            "content": [{"type": "input_text", "text": SYSTEM_PROMPT}]
        }
    ]

def chat_fn(user_message, history, messages_state):
    if messages_state is None:
        messages_state = build_initial_messages()

    # Add user message to state
    messages_state.append(
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_message}]
        }
    )

    # Call Responses API
    resp = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages_state,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True
    )

    assistant_message = resp.output_text

    # Add assistant message to state
    messages_state.append(
        {
            "role": "assistant",
            "content": [{"type": "output_text", "text": assistant_message}]
        }
    )

    # Update visible chat
    history = history + [(user_message, assistant_message)]

    return "", history, messages_state

def reset_chat():
    return [], build_initial_messages()

with gr.Blocks() as demo:
    # Header with logo at native size (no width/height restriction)
    gr.Markdown(
        f"""
<div style="display:flex; align-items:center; gap:12px;">
  <img src="{LOGO_URL}" />
  <div>
    <div style="font-size:22px; font-weight:600;">Python Chatbot</div>
    <div style="font-size:13px; opacity:0.8;">Powered by OpenAI Responses API</div>
  </div>
</div>
"""
    )

    chatbot = gr.Chatbot(height=420)

    input_box = gr.Textbox(
        placeholder="Type your Python question and press Enter",
        show_label=False
    )

    clear_btn = gr.Button("Clear")

    messages_state = gr.State(build_initial_messages())

    input_box.submit(
        fn=chat_fn,
        inputs=[input_box, chatbot, messages_state],
        outputs=[input_box, chatbot, messages_state]
    )

    clear_btn.click(
        fn=reset_chat,
        inputs=None,
        outputs=[chatbot, messages_state]
    )

demo.launch(debug=False)


/tmp/ipython-input-2076293516.py:109: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=420)
/tmp/ipython-input-2076293516.py:109: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(height=420)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c81ace7ebbbce29fc6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
import gradio as gr
from openai import OpenAI

client = OpenAI()

# Use the RAW GitHub URL so the image loads correctly
LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/nov25/main/logo_python.png"

SYSTEM_PROMPT = """Create an intelligent Python chatbot capable of engaging in natural, helpful, and contextually appropriate conversations with human users.

Requirements:
- Maintain conversational context over multiple user turns.
- Respond helpfully and accurately to a wide range of user inputs.
- Reason about user intent before generating each response.
- Politely ask clarifying questions if a request is ambiguous or unclear.
- Avoid hallucination or speculation—respond only with information you can justify or infer from context.
- If unable to answer, politely acknowledge the limitation.
- Please provide the answers only related to Python related questions and for any other questions answer as No

Process:
1. On each user message, first analyze prior context (if any) and what the user is likely asking/intending.
2. Think step-by-step (chain-of-thought) to determine the most relevant, helpful response. Always reason internally before presenting your answer.
3. If more information is needed, ask targeted clarifying questions.
4. Output your response, maintaining natural tone and conversational flow.
5. Continue the conversation until the user indicates they are finished.

Output:
- Each response should be in plain English, no markdown or code blocks unless explicitly requested.
- Maintain a single-paragraph, natural-sounding chat response of 1–3 sentences (unless a longer reply is requested or required).

Example—Instructions:
- Reasoning: "Recognize the user asked for Python list examples and may want to know how lists work."
- Conclusion/Output: "Sure! In Python, a list is a collection of items in a particular order. For example: my_list = [1, 2, 3, 4]. Would you like to see how to add or remove items?"

(For more advanced technical requests, reasoning steps and explanations may be slightly longer, but always conclude with a concise, clear reply to the user.)

Edge Cases & Important Considerations:
- If the user refers to prior conversation context, recall and incorporate it.
- Be warm, engaging, and never condescending.
- If asked for code, provide only what is needed and explain concisely.

REMINDER: Your primary objective is to serve as a helpful Python chatbot, reasoning about context before each response, and outputting clear, appropriate conversational replies.
"""

def build_initial_messages():
    return [
        {
            "role": "system",
            "content": [{"type": "input_text", "text": SYSTEM_PROMPT}]
        }
    ]

def chat_fn(user_message, history, messages_state):
    # Initialize state if needed
    if messages_state is None:
        messages_state = build_initial_messages()

    # Append user input
    messages_state.append(
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_message}]
        }
    )

    # Call Responses API
    resp = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages_state,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True
    )

    assistant_message = resp.output_text

    # Append assistant output
    messages_state.append(
        {
            "role": "assistant",
            "content": [{"type": "output_text", "text": assistant_message}]
        }
    )

    # Update visible chat history
    history = history + [(user_message, assistant_message)]

    # Clear textbox, update chat + state
    return "", history, messages_state

def reset_chat():
    return [], build_initial_messages()

with gr.Blocks() as demo:
    # Header with logo at native size (no constraints)
    gr.Markdown(
        f"""
<div style="display:flex; align-items:center; gap:12px;">
  <img src="{LOGO_URL}" />
  <div>
    <div style="font-size:22px; font-weight:600;">Python Chatbot</div>
    <div style="font-size:13px; opacity:0.8;">Powered by OpenAI Responses API</div>
  </div>
</div>
"""
    )

    chatbot = gr.Chatbot(height=420)
    input_box = gr.Textbox(
        placeholder="Type your Python question and press Enter",
        show_label=False
    )
    clear_btn = gr.Button("Clear")

    messages_state = gr.State(build_initial_messages())

    input_box.submit(
        fn=chat_fn,
        inputs=[input_box, chatbot, messages_state],
        outputs=[input_box, chatbot, messages_state]
    )

    clear_btn.click(
        fn=reset_chat,
        inputs=None,
        outputs=[chatbot, messages_state]
    )

demo.launch(debug=False)

/tmp/ipython-input-3217496571.py:112: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=420)
/tmp/ipython-input-3217496571.py:112: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(height=420)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5bc3d0be1eb0c1d005.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [26]:
import gradio as gr
from openai import OpenAI

client = OpenAI()

SYSTEM_PROMPT = """Create an intelligent Python chatbot capable of engaging in natural, helpful, and contextually appropriate conversations with human users.

Requirements:
- Maintain conversational context over multiple user turns.
- Respond helpfully and accurately to a wide range of user inputs.
- Reason about user intent before generating each response.
- Politely ask clarifying questions if a request is ambiguous or unclear.
- Avoid hallucination or speculation—respond only with information you can justify or infer from context.
- If unable to answer, politely acknowledge the limitation.
- Please reply only to the questions related to Quality Assurance, testing and for any other questions reply as "Approach Karthick for assistance"

Process:
1. On each user message, first analyze prior context (if any) and what the user is likely asking/intending.
2. Think step-by-step (chain-of-thought) to determine the most relevant, helpful response. Always reason internally before presenting your answer.
3. If more information is needed, ask targeted clarifying questions.
4. Output your response, maintaining natural tone and conversational flow.
5. Continue the conversation until the user indicates they are finished.

Output:
- Each response should be in plain English, no markdown or code blocks unless explicitly requested.
- Maintain a single-paragraph, natural-sounding chat response of 1–3 sentences (unless a longer reply is requested or required).

Example—Instructions:
- Reasoning: "Recognize the user asked for Python list examples and may want to know how lists work."
- Conclusion/Output: "Sure! In Python, a list is a collection of items in a particular order. For example: my_list = [1, 2, 3, 4]. Would you like to see how to add or remove items?"

(For more advanced technical requests, reasoning steps and explanations may be slightly longer, but always conclude with a concise, clear reply to the user.)

Edge Cases & Important Considerations:
- If the user refers to prior conversation context, recall and incorporate it.
- Be warm, engaging, and never condescending.
- If asked for code, provide only what is needed and explain concisely.

REMINDER: Your primary objective is to serve as a helpful Python chatbot, reasoning about context before each response, and outputting clear, appropriate conversational replies.
"""

def init_messages():
    return [
        {
            "role": "system",
            "content": [{"type": "input_text", "text": SYSTEM_PROMPT}]
        }
    ]

def respond(user_text, chat_history, messages):
    if messages is None:
        messages = init_messages()

    # add user turn
    messages.append(
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_text}]
        }
    )

    # call API
    response = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True
    )

    assistant_text = response.output_text

    # add assistant turn
    messages.append(
        {
            "role": "assistant",
            "content": [{"type": "output_text", "text": assistant_text}]
        }
    )

    # update UI chat history
    chat_history = chat_history + [(user_text, assistant_text)]

    return "", chat_history, messages


# -----------------------------
# Gradio UI (Upgraded)
# -----------------------------

FAQ_QUESTIONS = [
    "What is Quality Assurance?",
    "What is Software testing life cycle?",
    "What is Functional testing?",
    "What is Automation testing?",
    "What is Performance testing?",
    "What is API testing?",
    "What is User Acceptance?"
]

def set_question(q):
    return q

def clear_all():
    return [], init_messages(), ""

# Updated logo URL (RAW link)
LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/nov25/main/logo_python.png"

css = """
/* Overall spacing + subtle polish */
#app_container {max-width: 1200px; margin: 0 auto;}

/* Header styling */
.header-wrap {
    display: flex;
    align-items: center;
    gap: 14px;
    padding: 10px 6px 2px 6px;
}
.header-title {
    font-size: 28px;
    font-weight: 700;
    line-height: 1.1;
}
.header-subtitle {
    font-size: 12.5px;
    opacity: 0.75;
    margin-top: 2px;
}

/* FAQ card-like feel */
.faq-box {
    border: 1px solid rgba(255,255,255,0.08);
    border-radius: 12px;
    padding: 14px;
}

/* Make buttons full width in FAQ column */
.faq-btn button {
    width: 100%;
    justify-content: flex-start;
}
"""

with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
    # Header Row
    with gr.Row():
        with gr.Column(scale=1, min_width=80):
            logo = gr.Image(
                value=LOGO_URL,  # ✅ updated to raw URL
                label=None,
                show_label=False,
                show_download_button=False,
                show_share_button=False,
                height=64,
                width=64,
                container=False
            )
        with gr.Column(scale=10):
            gr.HTML(
                """
                <div class="header-wrap">
                    <div>
                        <div class="header-title">ENEC QA Plant Support Bot</div>
                        <div class="header-subtitle">
                            Ask anything related to ENEC QA Plant Support — operations, quality assurance, and guidelines.
                        </div>
                    </div>
                </div>
                """
            )

    gr.Markdown("---")

    # State (unchanged logic)
    state = gr.State(init_messages())

    # Two-column layout
    with gr.Row(equal_height=True):
        # LEFT: FAQ + Quick Ask
        with gr.Column(scale=4, min_width=320):
            with gr.Group(elem_classes=["faq-box"]):
                gr.Markdown("### FAQ — Most Asked Python Questions")

                gr.Markdown(
                    "Click a question to auto-fill it, then press **Enter** or click **Send**."
                )

                # Buttons for FAQ
                faq_buttons = []
                for q in FAQ_QUESTIONS:
                    b = gr.Button(q, elem_classes=["faq-btn"])
                    faq_buttons.append(b)

                gr.Markdown("### Quick prompt ideas")
                quick = gr.Radio(
                    choices=[
                        "Explain with a simple example",
                        "Give me a beginner-friendly analogy",
                        "Show common mistakes to avoid",
                        "Provide a short quiz question",
                        "Compare two approaches briefly"
                    ],
                    label="Add a style preference (optional)",
                    value=None
                )

        # RIGHT: Chat area
        with gr.Column(scale=8, min_width=520):
            chatbot = gr.Chatbot(
                height=520,
                bubble_full_width=False,
                label="Conversation"
            )

            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Type your Python question here…",
                    label=None,
                    scale=9
                )
                send = gr.Button("Send", variant="primary", scale=1)

            with gr.Row():
                clear = gr.Button("Clear Chat")
                gr.Markdown(
                    "<span style='opacity:0.7;font-size:12px;'>Context is preserved across turns unless you clear.</span>"
                )

    # FAQ button -> fill textbox
    for b, q in zip(faq_buttons, FAQ_QUESTIONS):
        b.click(
            fn=lambda q=q: set_question(q),
            inputs=None,
            outputs=msg
        )

    # Optional quick preference: append hint to textbox (UI-only)
    def apply_quick_pref(pref, current_text):
        if not pref:
            return current_text
        if current_text and current_text.strip():
            return f"{current_text.strip()} ({pref})"
        return pref

    quick.change(
        fn=apply_quick_pref,
        inputs=[quick, msg],
        outputs=msg
    )

    # Submit logic (unchanged)
    msg.submit(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    send.click(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    # Clear
    clear.click(
        fn=clear_all,
        inputs=None,
        outputs=[chatbot, state, msg]
    )

demo.launch(debug=False)

/tmp/ipython-input-1391761945.py:150: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
/tmp/ipython-input-1391761945.py:150: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
/tmp/ipython-input-1391761945.py:215: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-1391761945.py:215: DeprecationWarning: The 'bubble_full_widt

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://137ac14ddceccc171a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [27]:
import base64

def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

# Path to the uploaded image in Colab
local_image_path = '/content/ENEC logo.jpg'

try:
    base64_image_string = image_to_base64(local_image_path)
    print("Copy this Base64 string:\n")
    print(base64_image_string)
except FileNotFoundError:
    print(f"Error: The file '{local_image_path}' was not found. Please ensure the path is correct.")
except Exception as e:
    print(f"An error occurred: {e}")

Copy this Base64 string:

...


In [19]:
import base64

def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

# IMPORTANT: Replace this with the actual path to your local image file
local_image_path = 'C:/Users/Nithish/Desktop/ENEC logo.jpg' # Corrected path format

try:
    base64_image_string = image_to_base64(local_image_path)
    print("Copy this Base64 string:\n")
    print(base64_image_string)
except FileNotFoundError:
    print(f"Error: The file '{local_image_path}' was not found. Please ensure the path is correct.")
except Exception as e:
    print(f"An error occurred: {e}")

Error: The file 'C:/Users/Nithish/Desktop/ENEC logo.jpg' was not found. Please ensure the path is correct.


In [17]:
import base64

def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base66.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

# IMPORTANT: Replace this with the actual path to your local image file
local_image_path = 'C:/Users/Nithish/Desktop/ENEC logo.jpg' # Corrected path format

try:
    base64_image_string = image_to_base64(local_image_path)
    print("Copy this Base64 string:\n")
    print(base64_image_string)
except FileNotFoundError:
    print(f"Error: The file '{local_image_path}' was not found. Please ensure the path is correct.")
except Exception as e:
    print(f"An error occurred: {e}")

Error: The file 'C:/Users/Nithish/Desktop/ENEC logo.jpg' was not found. Please ensure the path is correct.


In [22]:
import base64

def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

# IMPORTANT: Replace this with the actual, correct path to your local image file
# For example: '/path/to/your/image.png' on Linux/macOS, or 'C:\Users\YourUser\Desktop\image.jpg' on Windows (use forward slashes or raw string 'r"C:\..."')
local_image_path = 'C:/Users/Nithish/Desktop/ENEC logo.jpg' # Example path

try:
    base64_image_string = image_to_base64(local_image_path)
    print("Copy this Base64 string:\n")
    print(base64_image_string)
except FileNotFoundError:
    print(f"Error: The file '{local_image_path}' was not found. Please ensure the path is correct.")
except Exception as e:
    print(f"An error occurred: {e}")

Error: The file 'C:/Users/Nithish/Desktop/ENEC logo.jpg' was not found. Please ensure the path is correct.


#Steps to deploy on Hugging Face Spaces (Gradio)

Hugging Face → New Space

Choose Gradio as SDK. This sets your Space to use Gradio via the README config.
Hugging Face

Pick a name and visibility.

Add files
1. app.py (your code).

2. requirements.txt.

requirements.txt
Use something like:

gradio
openai


##Add your secret

Go to your Space Settings → Variables and secrets.

Under Secrets, add:

Name: OPENAI_API_KEY

Value: your key
Secrets are injected into the runtime as environment variables.
Hugging Face


Commit

If you’re editing in the browser, just save/commit.

The Space will rebuild and launch automatically.

In [14]:


# ADD THIS IN APP.PY




import os
import gradio as gr
from openai import OpenAI

# -----------------------------
# Load OpenAI key from HF Secrets
# -----------------------------
# In Hugging Face Spaces, Secrets/Variables are available as environment variables.
# Add this in your Space Settings -> Variables and secrets -> Secrets
# Key name: OPENAI_API_KEY
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



client = OpenAI(api_key=OPENAI_API_KEY)

SYSTEM_PROMPT = """Create an intelligent Python chatbot capable of engaging in natural, helpful, and contextually appropriate conversations with human users.

Requirements:
- Maintain conversational context over multiple user turns.
- Respond helpfully and accurately to a wide range of user inputs.
- Reason about user intent before generating each response.
- Politely ask clarifying questions if a request is ambiguous or unclear.
- Avoid hallucination or speculation—respond only with information you can justify or infer from context.
- If unable to answer, politely acknowledge the limitation.

Process:
1. On each user message, first analyze prior context (if any) and what the user is likely asking/intending.
2. Think step-by-step (chain-of-thought) to determine the most relevant, helpful response. Always reason internally before presenting your answer.
3. If more information is needed, ask targeted clarifying questions.
4. Output your response, maintaining natural tone and conversational flow.
5. Continue the conversation until the user indicates they are finished.

Output:
- Each response should be in plain English, no markdown or code blocks unless explicitly requested.
- Maintain a single-paragraph, natural-sounding chat response of 1–3 sentences (unless a longer reply is requested or required).

Example—Instructions:
- Reasoning: "Recognize the user asked for Python list examples and may want to know how lists work."
- Conclusion/Output: "Sure! In Python, a list is a collection of items in a particular order. For example: my_list = [1, 2, 3, 4]. Would you like to see how to add or remove items?"

(For more advanced technical requests, reasoning steps and explanations may be slightly longer, but always conclude with a concise, clear reply to the user.)

Edge Cases & Important Considerations:
- If the user refers to prior conversation context, recall and incorporate it.
- Be warm, engaging, and never condescending.
- If asked for code, provide only what is needed and explain concisely.

REMINDER: Your primary objective is to serve as a helpful Python chatbot, reasoning about context before each response, and outputting clear, appropriate conversational replies.
"""

def init_messages():
    return [
        {
            "role": "system",
            "content": [{"type": "input_text", "text": SYSTEM_PROMPT}]
        }
    ]

def respond(user_text, chat_history, messages):
    if messages is None:
        messages = init_messages()

    # add user turn
    messages.append(
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_text}]
        }
    )

    # call API
    response = client.responses.create(
        model="gpt-5-chat-latest",
        input=messages,
        text={"format": {"type": "text"}},
        reasoning={},
        tools=[],
        temperature=1,
        max_output_tokens=2048,
        top_p=1,
        store=True
    )

    assistant_text = response.output_text

    # add assistant turn
    messages.append(
        {
            "role": "assistant",
            "content": [{"type": "output_text", "text": assistant_text}]
        }
    )

    # update UI chat history
    chat_history = chat_history + [(user_text, assistant_text)]

    return "", chat_history, messages


# -----------------------------
# Gradio UI (Upgraded)
# -----------------------------

FAQ_QUESTIONS = [
    "What is the difference between a list, tuple, and set in Python?",
    "How do I use dictionaries effectively in Python?",
    "What are Python functions and how do *args and **kwargs work?",
    "How does OOP work in Python (classes, objects, inheritance)?",
    "How do I handle errors using try/except?",
    "What are list comprehensions and when should I use them?",
    "How do I read and write files in Python?"
]

def set_question(q):
    return q

def clear_all():
    return [], init_messages(), ""

# Updated logo URL (RAW link)
LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/nov25/main/logo_python.png"

css = """
/* Overall spacing + subtle polish */
#app_container {max-width: 1200px; margin: 0 auto;}

/* Header styling */
.header-wrap {
    display: flex;
    align-items: center;
    gap: 14px;
    padding: 10px 6px 2px 6px;
}
.header-title {
    font-size: 28px;
    font-weight: 700;
    line-height: 1.1;
}
.header-subtitle {
    font-size: 12.5px;
    opacity: 0.75;
    margin-top: 2px;
}

/* FAQ card-like feel */
.faq-box {
    border: 1px solid rgba(255,255,255,0.08);
    border-radius: 12px;
    padding: 14px;
}

/* Make buttons full width in FAQ column */
.faq-btn button {
    width: 100%;
    justify-content: flex-start;
}
"""

with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
    # Header Row
    with gr.Row():
        with gr.Column(scale=1, min_width=80):
            logo = gr.Image(
                value=LOGO_URL,
                label=None,
                show_label=False,
                show_download_button=False,
                show_share_button=False,
                height=64,
                width=64,
                container=False
            )
        with gr.Column(scale=10):
            gr.HTML(
                """
                <div class="header-wrap">
                    <div>
                        <div class="header-title">Python Tutor Bot</div>
                        <div class="header-subtitle">
                            Ask anything about Python — concepts, debugging, best practices, and examples.
                        </div>
                    </div>
                </div>
                """
            )

    gr.Markdown("---")

    # State (unchanged logic)
    state = gr.State(init_messages())

    # Two-column layout
    with gr.Row(equal_height=True):
        # LEFT: FAQ + Quick Ask
        with gr.Column(scale=4, min_width=320):
            with gr.Group(elem_classes=["faq-box"]):
                gr.Markdown("### FAQ — Most Asked Python Questions")

                gr.Markdown(
                    "Click a question to auto-fill it, then press **Enter** or click **Send**."
                )

                # Buttons for FAQ
                faq_buttons = []
                for q in FAQ_QUESTIONS:
                    b = gr.Button(q, elem_classes=["faq-btn"])
                    faq_buttons.append(b)

                gr.Markdown("### Quick prompt ideas")
                quick = gr.Radio(
                    choices=[
                        "Explain with a simple example",
                        "Give me a beginner-friendly analogy",
                        "Show common mistakes to avoid",
                        "Provide a short quiz question",
                        "Compare two approaches briefly"
                    ],
                    label="Add a style preference (optional)",
                    value=None
                )

        # RIGHT: Chat area
        with gr.Column(scale=8, min_width=520):
            chatbot = gr.Chatbot(
                height=520,
                bubble_full_width=False,
                label="Conversation"
            )

            with gr.Row():
                msg = gr.Textbox(
                    placeholder="Type your Python question here…",
                    label=None,
                    scale=9
                )
                send = gr.Button("Send", variant="primary", scale=1)

            with gr.Row():
                clear = gr.Button("Clear Chat")
                gr.Markdown(
                    "<span style='opacity:0.7;font-size:12px;'>Context is preserved across turns unless you clear.</span>"
                )

    # FAQ button -> fill textbox
    for b, q in zip(faq_buttons, FAQ_QUESTIONS):
        b.click(
            fn=lambda q=q: set_question(q),
            inputs=None,
            outputs=msg
        )

    # Optional quick preference: append hint to textbox (UI-only)
    def apply_quick_pref(pref, current_text):
        if not pref:
            return current_text
        if current_text and current_text.strip():
            return f"{current_text.strip()} ({pref})"
        return pref

    quick.change(
        fn=apply_quick_pref,
        inputs=[quick, msg],
        outputs=msg
    )

    # Submit logic (unchanged)
    msg.submit(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    send.click(
        respond,
        inputs=[msg, chatbot, state],
        outputs=[msg, chatbot, state]
    )

    # Clear
    clear.click(
        fn=clear_all,
        inputs=None,
        outputs=[chatbot, state, msg]
    )

demo.launch(debug=False)


/tmp/ipython-input-1442312891.py:165: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
/tmp/ipython-input-1442312891.py:165: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft(), elem_id="app_container") as demo:
/tmp/ipython-input-1442312891.py:230: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-1442312891.py:230: DeprecationWarning: The 'bubble_full_widt

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1c6db252a8d5e4a07a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
